This notebook contains prealpha Haralick features calculation code along with time tests. <br>
**TODO: vectorization**

### Imports and constants

In [ ]:
import tensorflow as tf
import numpy as np

import cv2
import skimage.transform
from skimage.feature import greycomatrix, greycoprops

# time measuring 
import time
import pprint

In [ ]:
LOAD_PATH = '../examples/multispec_img.npy'
NUM_REP = 64

### Sample image loading

In [ ]:
img = np.load(LOAD_PATH)
print(f'img shape = {img.shape}')

picture shape = (65, 65, 8)


### Handcrafted features calculation

Features should be normalized therefore corresponding minima and maxima were precalculated.

In [ ]:
min_max_dict = {
  'min': {
    'arvi': -0.11833266519719478,
    'evi': -0.022776836311584298,
    'h_glcm_hom': 0.08442477027905304,
    'h_glcm_mean': 253.47205872093022,
    'h_glcm_sosv': 56737.77884884521,
    'h_mean': 19.422487037859074,
    'h_std': 1.4369563844979523,
    'i_glcm_mean': 83.60073284883721,
    'i_glcm_sosv': 14235.545471465104,
    'i_mean': 0.030918037472179714,
    'ndvi': -0.07979946801639441,
    'ndwi': -0.25679003333176903,
    's_glcm_correlation': 0.07081265512018585,
    's_glcm_mean': 7.709087693798451,
    's_glcm_sosv': 350.9815934669858,
    's_mean': 0.030918037472179714,
    'swir1_mean': 0.000253393665158371,
    'swir2_mean': 0.00021719457013574662,
    'tir1_mean': 0.0004084000464090962,
    'tir2_mean': 0.0004158255017983525},
 'max': {
    'arvi': 0.5567921249833387,
    'evi': 0.7661288947704729,
    'h_glcm_hom': 0.9988599263466074,
    'h_glcm_mean': 524.0095466085272,
    'h_glcm_sosv': 142741.5464050787,
    'h_mean': 289.8848410461291,
    'h_std': 139.78125522930878,
    'i_glcm_mean': 133.80282829457366,
    'i_glcm_sosv': 17622.45531356966,
    'i_mean': 0.9989349490751622,
    'ndvi': 0.6946931383411229,
    'ndwi': 0.4088313329186157,
    's_glcm_correlation': 0.9709833637974503,
    's_glcm_mean': 105.0412703488372,
    's_glcm_sosv': 10330.89298209496,
    's_mean': 0.9989349490751622,
    'swir1_mean': 0.45185752407471874,
    'swir2_mean': 0.3782851838960436,
    'tir1_mean': 0.8493412228796844,
    'tir2_mean': 0.8320232045480914}}

In [ ]:
def rgb2hsi(red, green, blue): # input: normalized RGB
  # output:
  # H - Hue [0, 360]
  # S - Saturation [0, 1]
  # I - Intensity [0, 255]
  
  temp = 0.5 * (red - green + red - blue) / (np.sqrt((red - green) * (red - green) + (red - blue) * (green - blue) + 0.001))
  temp = np.arccos(temp)
  temp2 = green >= blue
  temp2 = np.multiply(temp, temp2)
  temp3 = green < blue
  temp4 = 2 * np.pi - temp
  temp4 = np.multiply(temp3, temp4)

  H = (temp2 + temp4) * 180 / np.pi
  S = (1 - 3 * (np.minimum(np.minimum(red, green), blue))/(red+ g reen + blue + 0.001))
  I = ((red + green + blue) / 3) * 255
  
  return H,S,I

In [ ]:
def create_glcm(H, S, I): # input: HSI
  # output: GLCM matrices

  H_GLCM = greycomatrix(image = H.astype(int), distances = [1], 
                        angles = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4], levels = 361)
  S_GLCM = greycomatrix(image = (S * 100).astype(int), distances = [1], 
                        angles = [0, np.pi / 4, np.pi / 2, 3 * np.pi / 4], levels = 101)
  I_GLCM = greycomatrix(image = (np.round_(I)).astype(int), distances = [1], 
                        angles=[0, np.pi / 4, np.pi / 2, 3 * np.pi / 4], levels = 256)

  return H_GLCM, S_GLCM, I_GLCM

In [ ]:
def mean_GLCM(H_GLCM, S_GLCM, I_GLCM): # input: GLCM matrices
  # output: GLCM means

  H_GLCM = np.squeeze(H_GLCM)
  H_GLCM = np.mean(H_GLCM, axis  =2)

  S_GLCM = np.squeeze(S_GLCM)
  S_GLCM = np.mean(S_GLCM, axis = 2)

  I_GLCM = np.squeeze(I_GLCM)
  I_GLCM = np.mean(I_GLCM, axis = 2)

  return H_GLCM, S_GLCM, I_GLCM

In [ ]:
elapsed_times = {} # elapsed time will be measured and averaged for each section denoted by a proper comment

for _ in range(NUM_REP):
  
  # channels slicing
  start = time.time()

  R = img[:, :, 0]
  G = img[:, :, 1]
  B = img[:, :, 2] 
  NIR = img[:, :, 3]
  SWIR1 = img[:, :, 4]
  SWIR2 = img[:, :, 5] 
  TIR1 = img[:, :, 6]
  TIR2 = img[:, :, 7]

  end = time.time() - start
  elapsed_times.setdefault('channels slicing', 0)
  elapsed_times['channels slicing'] += end

  # HSI 
  start = time.time()

  H, S, I = rgb2hsi(R, G, B) 

  end = time.time() - start
  elapsed_times.setdefault('HSI', 0)
  elapsed_times['HSI'] += end

  # GLCM
  start = time.time()

  H_GLCM_Org, S_GLCM_Org, I_GLCM_Org = create_glcm(H, S, I) # 4 matrices for each band (one per direction)
  H_GLCM, S_GLCM, I_GLCM = mean_GLCM(H_GLCM_Org, S_GLCM_Org, I_GLCM_Org) 
  H_GLCM_prob = (H_GLCM / np.sum(H_GLCM)) + 0.00001 # for numeric stability
  S_GLCM_prob = (S_GLCM / np.sum(S_GLCM)) + 0.00001
  I_GLCM_prob = (I_GLCM / np.sum(I_GLCM)) + 0.00001

  end = time.time() - start
  elapsed_times.setdefault('GLCM', 0)
  elapsed_times['GLCM'] += end

  # *VI
  start = time.time()

  arvi = ((NIR - (2 * R - B)) / (NIR + (2 * R + B) + 0.0001)).mean() 
  arvi = (arvi - min_max_dict['min']['arvi']) / (min_max_dict['max']['arvi'] - min_max_dict['min']['arvi'])
  evi = (2.5 * ((NIR - R) / (NIR + (R) * 6 - (B) * 7.5 + 1 + 0.0001))).mean()
  evi = (evi - min_max_dict['min']['evi']) / (min_max_dict['max']['evi'] - min_max_dict['min']['evi'])
  ndvi = ((NIR - R)/((NIR + R) + 0.0001)).mean()
  ndvi = (ndvi - min_max_dict['min']['ndvi']) / (min_max_dict['max']['ndvi'] - min_max_dict['min']['ndvi'])
  ndwi = ((NIR - SWIR1)/(NIR + SWIR1 + 0.0001)).mean()
  ndwi = (ndwi - min_max_dict['min']['ndwi']) / (min_max_dict['max']['ndwi'] - min_max_dict['min']['ndwi'])

  end = time.time() - start
  elapsed_times.setdefault('ARVI/EVI/NDVI/NDWI', 0)
  elapsed_times['ARVI/EVI/NDVI/NDWI'] += end

  # bands normalization
  start = time.time()

  H_std = H.std()
  H_std = (H_std - min_max_dict['min']['h_std']) / (min_max_dict['max']['h_std'] - min_max_dict['min']['h_std'])
  H_mean = H.mean()
  H_mean = (H_mean - min_max_dict['min']['h_mean']) / (min_max_dict['max']['h_mean'] - min_max_dict['min']['h_mean'])
  S_mean = S.mean()
  S_mean = (S_mean - min_max_dict['min']['s_mean']) / (min_max_dict['max']['s_mean'] - min_max_dict['min']['s_mean'])
  I_mean = S.mean()
  I_mean = (I_mean - min_max_dict['min']['i_mean']) / (min_max_dict['max']['i_mean'] - min_max_dict['min']['i_mean'])
  TIR1_mean = TIR1.mean()
  TIR1_mean = (TIR1_mean - min_max_dict['min']['tir1_mean']) / (min_max_dict['max']['tir1_mean'] - min_max_dict['min']['tir1_mean'])
  TIR2_mean = TIR2.mean()
  TIR2_mean = (TIR2_mean - min_max_dict['min']['tir2_mean']) / (min_max_dict['max']['tir2_mean'] - min_max_dict['min']['tir2_mean'])
  SWIR1_mean = SWIR1.mean()
  SWIR1_mean = (SWIR1_mean - min_max_dict['min']['swir1_mean']) / (min_max_dict['max']['swir1_mean'] - min_max_dict['min']['swir1_mean'])
  SWIR2_mean = SWIR2.mean()
  SWIR2_mean = (SWIR2_mean - min_max_dict['min']['swir2_mean']) / (min_max_dict['max']['swir2_mean'] - min_max_dict['min']['swir2_mean'])

  end = time.time() - start
  elapsed_times.setdefault('norm', 0)
  elapsed_times['norm'] += end

  # H_GLCM
  start = time.time()

  H_GLCM_mean = np.array(H_GLCM_prob * np.array(range(H_GLCM_prob.shape[1]))).sum()
  H_GLCM_mean = (H_GLCM_mean - min_max_dict['min']['h_glcm_mean']) / (min_max_dict['max']['h_glcm_mean'] - min_max_dict['min']['h_glcm_mean'])
  raw = np.empty_like(H_GLCM_prob)
  raw[...] = np.arange(H_GLCM_prob.shape[0])
  raw = np.transpose(raw)
  minus_mean = (raw - H_GLCM_prob.mean()) ** 2
  H_GLCM_sosv = np.apply_over_axes(np.sum, minus_mean * H_GLCM_prob, axes = (0, 1))[0][0]
  H_GLCM_sosv = (H_GLCM_sosv - min_max_dict['min']['h_glcm_sosv']) / (min_max_dict['max']['h_glcm_sosv'] - min_max_dict['min']['h_glcm_sosv']) 
  H_GLCM_Homogeneity = (greycoprops(P = H_GLCM_Org, prop = 'homogeneity')).mean()
  H_GLCM_Homogeneity = (H_GLCM_Homogeneity - min_max_dict['min']['h_glcm_hom']) / (min_max_dict['max']['h_glcm_hom'] - min_max_dict['min']['h_glcm_hom']) 

  end = time.time() - start
  elapsed_times.setdefault('H_GLCM', 0)
  elapsed_times['H_GLCM'] += end

  # S_GLCM
  start = time.time()

  S_GLCM_mean = np.array(S_GLCM_prob * np.array(range(S_GLCM_prob.shape[1]))).sum()
  S_GLCM_mean = (S_GLCM_mean - min_max_dict['min']['s_glcm_mean']) / (min_max_dict['max']['s_glcm_mean'] - min_max_dict['min']['s_glcm_mean'])
  raw = np.empty_like(S_GLCM_prob)
  raw[...] = np.arange(S_GLCM_prob.shape[0])
  raw = np.transpose(raw)
  minus_mean = (raw - S_GLCM_prob.mean()) ** 2
  S_GLCM_sosv = np.apply_over_axes(np.sum, minus_mean * S_GLCM_prob, axes = (0, 1))[0][0] 
  S_GLCM_sosv = (S_GLCM_sosv - min_max_dict['min']['s_glcm_sosv']) / (min_max_dict['max']['s_glcm_sosv'] - min_max_dict['min']['s_glcm_sosv'])
  S_GLCM_Correlation = (greycoprops(P = S_GLCM_Org, prop = 'correlation')).mean()
  S_GLCM_Correlation = (S_GLCM_Correlation - min_max_dict['min']['s_glcm_correlation']) / (min_max_dict['max']['s_glcm_correlation'] - min_max_dict['min']['s_glcm_correlation'])

  end = time.time() - start
  elapsed_times.setdefault('S_GLCM', 0)
  elapsed_times['S_GLCM'] += end

  # I_GLCM
  start = time.time()

  I_GLCM_mean = np.array(I_GLCM_prob * np.array(range(I_GLCM_prob.shape[1]))).sum()
  I_GLCM_mean = (I_GLCM_mean - min_max_dict['min']['i_glcm_mean']) / (min_max_dict['max']['i_glcm_mean'] - min_max_dict['min']['i_glcm_mean'])
  raw = np.empty_like(I_GLCM_prob)
  raw[...] = np.arange(I_GLCM_prob.shape[0])
  raw = np.transpose(raw)
  minus_mean = (raw - I_GLCM_prob.mean()) ** 2
  I_GLCM_sosv = np.apply_over_axes(np.sum, minus_mean * I_GLCM_prob, axes=(0, 1))[0][0] 
  I_GLCM_sosv = (I_GLCM_sosv - min_max_dict['min']['i_glcm_sosv']) / (min_max_dict['max']['i_glcm_sosv'] - min_max_dict['min']['i_glcm_sosv'])

  end = time.time() - start
  elapsed_times.setdefault('I_GLCM', 0)
  elapsed_times['I_GLCM'] += end

  features = [arvi, evi, ndvi, ndwi, H_std, H_mean, S_mean, I_mean, TIR1_mean, TIR2_mean, SWIR1_mean, SWIR2_mean, H_GLCM_mean, 
              H_GLCM_sosv, H_GLCM_Homogeneity, S_GLCM_mean, S_GLCM_sosv, S_GLCM_Correlation, I_GLCM_mean, I_GLCM_sosv]

  # assigning
  start = time.time()

  arr = np.array(features, dtype = np.float32)
  ret_arr = np.zeros((65, 65, 1), dtype = np.float32)
  ret_arr[0, 0 : 20, 0] = arr

  end = time.time() - start
  elapsed_times.setdefault('assign', 0)
  elapsed_times['assign'] += end

# means and total time calculation
elapsed_means = {}
total_time = 0

for key, value in elapsed_times.items():
  elapsed_means[key] = value / float(NUM_REP)
  total_time += value

print('Mean elapsed times (seconds):')
pprint.pprint(elapsed_means)

print(f'\nTotal time for {NUM_REP} iterations: {total_time}s')

print('\nCalculated features:')
pprint.pprint(features)

Mean elapsed times (seconds):
{'ARVI/EVI/NDVI/NDWI': 0.00019965693354606628,
 'GLCM': 0.006812989711761475,
 'HSI': 0.00019410252571105957,
 'H_GLCM': 0.005897361785173416,
 'I_GLCM': 0.0007641203701496124,
 'S_GLCM': 0.0013883747160434723,
 'assign': 1.6842037439346313e-05,
 'channels slicing': 4.649162292480469e-06,
 'norm': 0.00013906508684158325}

Total time for 64 iterations: 0.9866983890533447s

Calculated features:
[0.583294407539082,
 0.4205302594307736,
 0.6268935771851192,
 0.530733633409135,
 0.7037631531148164,
 0.5673630504946428,
 0.09887176630078556,
 0.09887176630078556,
 0.9261081124994331,
 0.9295599175018712,
 0.47112820685939716,
 0.37422085828175966,
 0.5644887136328478,
 0.4509478981206715,
 0.12314932605348573,
 0.09872824486093075,
 0.024813594090530272,
 0.898877097358546,
 0.5086731561023332,
 0.2263605138691972]
